Прежде чем проверять задания:
- **Перезапустите ядро** (**restart the kernel**) (В меню, выбрать Ядро (Kernel) $\rightarrow$ Перезапустить (Restart)
- Затем **Выполнить все ячейки**  **run all cells** (В меню, выбрать Ячейка (Cell) $\rightarrow$ Запустить все (Run All).

Убедитесь, что заполнены все ячейки с комментарием "НАЧАЛО ВАШЕГО РЕШЕНИЯ".

После ячеек с заданием следуют ячейки с проверкой с помощью assert.

Если в коде есть ошибки, assert выведет уведомление об ошибке.

Если в коде нет ошибок, assert отработает без вывода дополнительной информации.

---

# Цель занятия
На этом занятии мы на практике закрепим работу с решающими деревьями, композициями деревьев на основе бэггинга и случайного леса.

**Решающее дерево** - это алгоритм машинного обучения, который используется для решения задач классификации и регрессии. Оно представляет собой древовидную структуру, где каждый узел представляет тест на одном из признаков, а каждая ветвь - возможный результат этого теста. Листья дерева представляют собой конечный результат - прогноз для новых данных.

В процессе построения решающего дерева, алгоритм выбирает тест, который лучше всего разделяет данные на различные классы или предсказывает значение целевой переменной. Затем данные разбиваются на две или более частей в соответствии с результатами теста. Этот процесс повторяется для каждой полученной части, пока не будет достигнут критерий останова.

In [1]:
# импорт необходимых библиотек

import numpy as np

## Часть 1. Задание 1. Реализация критерия Джини.

Функция gini_index принимает вектор y с дискретными значениями и вычисляет значение критерия Джини для данного вектора. В функции gini_index сначала подсчитывается количество уникальных значений в векторе y, затем вычисляется вероятность каждого уникального значения и, наконец, вычисляется значение критерия Джини. Отличие заключается в том, что значение критерия Джини вычисляется по формуле 1 - сумма квадратов вероятностей.

In [2]:
y = np.array([0, 0, 1])
_, counts = np.unique(y, return_counts=True)
probs = counts / len(y)
probs

array([0.66666667, 0.33333333])

In [3]:
# Завершите реализацию функции gini_index 

def gini_index(y):

    # Вычисляет критерий Джини для вектора y со значениями дискретных переменных.

    # Аргументы:
    # - y: вектор numpy с дискретными значениями.

    # Возвращает:
    # - gini: значение критерия Джини типа float.

    # Подсчитываем количество каждого уникального значения в y.
    _, counts = np.unique(y, return_counts=True)
    # Вычисляем вероятность каждого уникального значения.
    probs = counts / len(y)
    
    if not len(probs):
        return 0
    
    """
    Вычислите значение критерия Джини и запишите его в переменную criterion
    Формула критерия Джини: 1 - сумма квадратов вероятностей  
    Пример для энтропии: criterion = -np.sum(probs * np.log2(probs)) !!! заменить на формулу для Джини
    """
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    criterion = 1 -  np.sum(np.square(probs))
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ
    
    return criterion

In [4]:
# Пустой вектор: 
assert gini_index(np.array([])) == 0
# Вектор с одним элементом:
assert gini_index(np.array([1])) == 0
# Вектор с двумя одинаковыми элементами: 
assert gini_index(np.array([2,2])) == 0
# Вектор с двумя разными элементами: 
assert gini_index(np.array([1,2])) == 0.5
# Вектор с тремя одинаковыми элементами: 
assert gini_index(np.array([0,0,0])) == 0
# Вектор с тремя элементами, два из которых одинаковые: 
assert gini_index(np.array([0,0,1])) == 0.4444444444444444
# Вектор с тремя элементами, все элементы разные: 
assert gini_index(np.array([1,2,3])) == 0.6666666666666667
# Вектор с четырьмя одинаковыми элементами: 
assert gini_index(np.array([7,7,7,7])) == 0
# Вектор с четырьмя элементами, два из которых одинаковые: 
assert gini_index(np.array([5,5,2,1])) == 0.625
# Вектор с пятью элементами, все элементы разные:
assert np.isclose(gini_index(np.array([5,4,3,2,1])), 0.7999999999999998, atol=0.1)

## Часть 1.  Задание 2. Реализация прироста информации на основе критерия Джини.

Для реализации прироста информации на основе критерия Джини для вектора признаков нужно вычислить критерий Джини для каждого разбиения и выбрать разбиение с наименьшим критерием Джини. 

In [5]:
def find_best_split_gini(X, y):

    # Находит лучшее разбиение для вектора признаков X и вектора целевой переменной y, используя критерий Джини.

    # Аргументы:
    # - X: вектор numpy с вещественными значениями признаков.
    # - y: вектор numpy с дискретными значениями целевой переменной.

    # Возвращает:
    # - best_feature: индекс признака, по которому было найдено лучшее разбиение.
    # - best_threshold: значение порога, по которому было найдено лучшее разбиение.
    # - best_gain: значение критерия энтропии для лучшего разбиения.

    best_feature, best_threshold, best_gain = None, None, 0
    # Итерируемся по всем признакам.
    for feature in range(X.shape[1]):
        # Находим уникальные значения признака.
        thresholds = np.unique(X[:, feature])
        # Итерируемся по всем возможным пороговым значениям признака.
        for threshold in thresholds:
            # Определяем индексы объектов, которые относятся к левому поддереву и правому поддереву.
            left_indices = X[:, feature] <= threshold
            right_indices = X[:, feature] > threshold
            # Пропускаем текущую итерацию, если не найдены объекты, которые относятся к левому или правому поддереву.
            if len(left_indices) == 0 or len(right_indices) == 0:
                continue
            # Определяем вектор целевой переменной для объектов, которые относятся к левому и правому поддереву.
            left_y, right_y = y[left_indices], y[right_indices]
            """
            Вычисляем значение прироста информации для текущего разбиения.
            Необходимо сохранить результат вычисления в переменную gain
            Пример для энтропии: 
            gain = entropy(y) - (len(left_y) / len(y)) * entropy(left_y) - (len(right_y) / len(y)) * entropy(right_y) 
            !!! заменить на вычисление критерия Джини
            """
            # НАЧАЛО ВАШЕГО РЕШЕНИЯ
            gain = gini_index(y) - (len(left_y) / len(y)) * gini_index(left_y) - (len(right_y) / len(y) * gini_index(right_y))
            # КОНЕЦ ВАШЕГО РЕШЕНИЯ
            # Обновляем значения лучшего разбиения, если найдено разбиение с большим значением
            if gain > best_gain:
                best_feature, best_threshold, best_gain = feature, threshold, gain
    return best_feature, best_threshold, best_gain

In [6]:
# Проверим, что функция `find_best_split_gini` работает правильно на примере:

X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]])
y = np.array([0, 1, 1, 0, 1])

best_feature, best_threshold, best_gain = find_best_split_gini(X, y)


assert best_feature == 0
assert best_threshold == 1
assert round(best_gain, 2) == 0.18


# Проверим, что функция `find_best_split_gini` работает правильно на примере, когда все элементы одного класса:

X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]])
y = np.array([0, 0, 0, 0, 0])

best_feature, best_threshold, best_gain = find_best_split_gini(X, y)

assert best_feature is None
assert best_threshold is None
assert best_gain == 0


# Проверим, что функция `find_best_split_gini` работает правильно на примере, когда все признаки одинаковы:

X = np.array([[1, 1], [1, 1], [1, 1], [1, 1], [1, 1]])
y = np.array([0, 1, 1, 0, 1])

best_feature, best_threshold, best_gain = find_best_split_gini(X, y)

assert best_feature is None
assert best_threshold is None
assert best_gain == 0


# Проверим, что функция `find_best_split_gini` работает правильно на примере, когда все элементы разных классов:

X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]])
y = np.array([0, 1, 0, 1, 0])

best_feature, best_threshold, best_gain = find_best_split_gini(X, y)

assert best_feature == 0
assert best_threshold == 1
assert round(best_gain, 2) == 0.08

## Часть 2. Задание 3. Бэггинг для классификации

В этом примере мы воспользуемся набором данных Iris.

Нужно будет определить базовый оценщик как решающее дерево классификации, определить BaggingClassifier с количеством оценщиков n_estimators равным 10. Затем мы обучим модель на обучающем наборе данных и оценим ее производительность на тестовом наборе данных с помощью метрики точности (accuracy_score).

In [7]:
# Загрузка библиотек
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Загрузка датасета
iris = load_iris()
X, y = iris.data, iris.target

# Разделение набора данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
"""
Инициализируйте бэггинг-классификатор на основе класса BaggingClassifier, 
в параметр estimator передайте значение DecisionTreeClassifier(), 
в параметр n_estimators - значение 30, 
в параметр random_state - значение 0.
Экземпляр сохраните в переменную bagging

Пример с другой моделью:
bagging = BaggingClassifier(estimator=SVC(), n_estimators=10, random_state=1)
"""
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
bagging = BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=30, random_state=0)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [9]:
assert type(bagging) == BaggingClassifier

In [10]:
# Обучение бэггинг-классификатора на обучающем наборе
bagging.fit(X_train, y_train)

# Оценка производительности бэггинг-классификатора на тестовом наборе
y_pred = bagging.predict(X_test)

In [11]:
"""
Посчитайте f1_macro метрику для трех классов и сохраните в переменную f1_macro
Пример для f1_micro:
f1_micro = f1_score(y_test, y_pred, average='micro')
"""
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
f1_macro = f1_score(y_test, y_pred, average='macro')
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

print("f1_macro:", f1_macro)

f1_macro: 1.0


In [12]:
assert f1_macro > 0.95

## Часть 3. Задание 4. Случайный лес для регрессии

Случайный лес (Random Forest) - это алгоритм машинного обучения, который является комбинацией множества решающих деревьев. Он применяется как для задач классификации, так и для задач регрессии.

Суть метода заключается в том, что мы строим несколько деревьев решений на случайных подмножествах данных и случайных подмножествах признаков на каждом нелистовом узле, а затем усредняем их ответы для уменьшения эффекта переобучения. Для каждого дерева в случайном лесу используется только подмножество данных, которое выбирается случайным образом с возвращением (bootstrap).

Кроме того, для каждого разбиения дерева в случайном лесу выбирается только подмножество признаков, которые можно использовать для разделения узлов. Это позволяет получить более разнообразные деревья и уменьшить вероятность переобучения.

In [13]:
# загрузка библиотек
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [14]:
# загрузка датасета
df = pd.read_csv("Boston.csv", header=0, index_col=0)
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [15]:
# Разделим данные на тренировочную и тестовую выборки:
X = df.drop('medv', axis=1)
y = df['medv']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
"""
Инициализируйте экземпляр класса RandomForestRegressor, 
в параметр n_estimators передайте значение 100, 
в параметр max_depth - значение 5, 
в параметр random_state - значение 0.
Экземпляр сохраните в переменную rfr

Пример с другой моделью:
rfc = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
"""

# НАЧАЛО ВАШЕГО РЕШЕНИЯ
rfr = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=0)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [17]:
assert type(rfr) == RandomForestRegressor

In [18]:
rfr.fit(X_train, y_train)
# Оценим качество модели на тестовой выборке с использованием метрики MAE:
y_pred = rfr.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)

MAE: 2.218644233909917
